<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#FUNCTIONAL-CODE" data-toc-modified-id="FUNCTIONAL-CODE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>FUNCTIONAL CODE</a></span></li></ul></div>

# FUNCTIONAL CODE

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time
import requests
from bs4 import BeautifulSoup

import datetime
from dateutil.relativedelta import *
from dateutil import rrule


CPV = ["48000000-8","73000000-2","72000000-5","72100000-6","72200000-7","72300000-8","72400000-4","72500000-0","72600000-6","72700000-7","72800000-8","72900000-9"]

#CPV = ["72100000-6","72200000-7","72300000-8","72400000-4","72500000-0","72600000-6","72700000-7","72800000-8","72900000-9"]

#month diffence
def diff_month(d1, d2):
    return -1*((d1.year - d2.year) * 12 + d1.month - d2.month)


def diff_weeks(start_date, end_date):
    weeks = rrule.rrule(rrule.WEEKLY, dtstart=start_date, until=end_date)
    return weeks.count()

#ano mes dia hora minutos segundos
data_inicial = datetime.datetime(2018, 1, 1, 0, 0, 0)
data_final = datetime.datetime.now()

#accessing Chromedriver
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
    
    # Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", {
     "profile.default_content_setting_values.notifications": 1
})
    # chromedriver should be in the same folder as file
browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=option)

#contratos (file_number)
file_number = -1

#month skipper
skipper = 1

for cpv in CPV:
    print(file_number)
    for i in range(diff_weeks(data_inicial,data_final)):
        #boundaries
        curr_floor_boundary = (data_inicial+relativedelta(weeks=+i)).strftime('%Y-%m-%d')
        curr_ceiling_boundary = (data_inicial+relativedelta(weeks=+(i+skipper))).strftime('%Y-%m-%d')
        
        #while break
        next_ = True
        print("Doing ("+ cpv + ") " + curr_floor_boundary + "-" +  curr_ceiling_boundary )
        while(next_):
            try:
                browser.implicitly_wait(5)
                
                #if in_ == False:
                browser.get('https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&texto=&tipo=0&tipocontrato=0&cpv='+cpv+'&aqinfo=&adjudicante=&adjudicataria=&sel_price=price_c1&desdeprecocontrato=&ateprecocontrato=&desdeprecoefectivo=&ateprecoefectivo=&sel_date=date_c1&desdedatacontrato='+curr_floor_boundary+'&atedatacontrato='+ curr_ceiling_boundary+'&desdedatapublicacao=&atedatapublicacao=&desdeprazoexecucao=&ateprazoexecucao=&desdedatafecho=&atedatafecho=&pais=0&distrito=0&concelho=0')  
                #browser.get('https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&texto=&tipo=0&tipocontrato=0&cpv=48000000-8&aqinfo=&adjudicante=&adjudicataria=&sel_price=price_c1&desdeprecocontrato=&ateprecocontrato=&desdeprecoefectivo=&ateprecoefectivo=&sel_date=date_c1&desdedatacontrato=2023-02-05&atedatacontrato=2023-02-05&desdedatapublicacao=&atedatapublicacao=&desdeprazoexecucao=&ateprazoexecucao=&desdedatafecho=&atedatafecho=&pais=0&distrito=0&concelho=0')  

                # W8 for AJAX to run
                #xPath
                #element = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH, "/html[1]/body[1]/div[2]/div[1]/div[1]/div[1]/div[2]/div[3]/a[1]")))
                element = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH, "/html[1]/body[1]/div[2]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]")))
                #W8 for page to finish loading

                #Get page information
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                result_tag = soup.find('div', {'class': 'alert alert-info'}) 
                tag_strip = list(result_tag.stripped_strings)
                total_results = tag_strip[0]
                total_contracts = int(total_results[slice(len(total_results)-1,len(total_results))])

                #have a total of single digit contracts
                if len(total_results) == 23:
                    if total_contracts == 0:
                        #print("got 0 records")
                        next_ = False
                        pass

                #>=24 double or superior digits
                browser.find_element_by_xpath('//body/div[2]/div[1]/div[1]/div[2]/div[2]/div[3]/a[1]').click()
                #print("got it?")
                #W8 for contratos.csv to finsh downloading
                time.sleep(15)
                next_ = False
                file_number = file_number + 1
            except:
                print("error found 0 records or could not load any records")
                pass
print("Done")
browser.close()

Function Testing

In [ ]:
import datetime
from dateutil.relativedelta import *
def diff_month(d1, d2):
    return -1*((d1.year - d2.year) * 12 + d1.isocalendar()[1] - d2.isocalendar()[1])


data_inicial = datetime.datetime(2018, 1, 1, 0, 0, 0)
data_final = datetime.datetime.now()


diff_month(data_inicial,data_final)

In [ ]:
from dateutil import rrule
import datetime

def weeks_between(start_date, end_date):
    weeks = rrule.rrule(rrule.WEEKLY, dtstart=start_date, until=end_date)
    return weeks.count()
data_inicial = datetime.datetime(2018, 1, 1, 0, 0, 0)
data_final = datetime.datetime.now()

weeks_between(data_inicial,data_final)



OLD CODE

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time
import requests
from bs4 import BeautifulSoup

import datetime
from dateutil.relativedelta import *


CPV = ["48000000-8","73000000-2","72000000-5","72100000-5","72200000-7","72300000-8",
       "72400000-4","72500000-0","72600000-6","72700000-7","72800000-8","72900000-9"]

#month diffence
def diff_month(d1, d2):
    return -1*((d1.year - d2.year) * 12 + d1.month - d2.month)


#ano mes dia hora minutos segundos
data_inicial = datetime.datetime(2018, 1, 1, 0, 0, 0)

#ano mes dia
data_inicio = ["2023-01-01","2023-02-01"]
data_fim= ["2023-02-01","2023-03-01"]

#accessing Chromedriver
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
    
    # Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", {
     "profile.default_content_setting_values.notifications": 1
})
    # chromedriver should be in the same folder as file
browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=option)


for cpv in CPV:
    for i in range(len(data_inicio)):
        in_= False
        next_ = True
        dInicio = data_inicio[i]
        dFim = data_fim[i]
        print("Doing ("+ cpv + ") " + data_inicio[i] + "-" + data_fim[i] )
        while(next_):
            try:
                browser.implicitly_wait(5)
                
                #if in_ == False:
                browser.get('https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&texto=&tipo=0&tipocontrato=0&cpv='+cpv+'&aqinfo=&adjudicante=&adjudicataria=&sel_price=price_c1&desdeprecocontrato=&ateprecocontrato=&desdeprecoefectivo=&ateprecoefectivo=&sel_date=date_c1&desdedatacontrato='+data_inicio[i]+'&atedatacontrato='+data_fim[i]+'&desdedatapublicacao=&atedatapublicacao=&desdeprazoexecucao=&ateprazoexecucao=&desdedatafecho=&atedatafecho=&pais=0&distrito=0&concelho=0')  
                #browser.get('https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&texto=&tipo=0&tipocontrato=0&cpv=48000000-8&aqinfo=&adjudicante=&adjudicataria=&sel_price=price_c1&desdeprecocontrato=&ateprecocontrato=&desdeprecoefectivo=&ateprecoefectivo=&sel_date=date_c1&desdedatacontrato=2023-02-05&atedatacontrato=2023-02-05&desdedatapublicacao=&atedatapublicacao=&desdeprazoexecucao=&ateprazoexecucao=&desdedatafecho=&atedatafecho=&pais=0&distrito=0&concelho=0')  

                # W8 for AJAX to run
                #xPath
                #element = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH, "/html[1]/body[1]/div[2]/div[1]/div[1]/div[1]/div[2]/div[3]/a[1]")))
                element = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH, "/html[1]/body[1]/div[2]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]")))
                #W8 for page to finish loading

                #Get page information
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                result_tag = soup.find('div', {'class': 'alert alert-info'}) 
                tag_strip = list(result_tag.stripped_strings)
                total_results = tag_strip[0]
                #print(len(total_results))
                total_contracts = int(total_results[slice(len(total_results)-1,len(total_results))])


                #have a total of single digit contracts
                if len(total_results) == 23:
                    if total_contracts == 0:
                        #print("got 0 records")
                        next_ = False
                        pass

                #>=24 double or superior digits
                browser.find_element_by_xpath('//body/div[2]/div[1]/div[1]/div[1]/div[2]/div[3]/a[1]').click()
                #W8 for contratos.csv to finsh downloading
                time.sleep(15)
                next_ = False
            except:
                pass
            
browser.close()

In [ ]:
from bs4 import BeautifulSoup
import requests 
page = requests.get("https://www.govis.pt/codigos/cpv/72100000-6")
soup = BeautifulSoup(page.content, 'html.parser')
soup

In [ ]:
soup.find('p')